<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 1: Importing Library</h1>
</div>

In [1]:
#Step 1. Importing Libraries
import sys
import math
import bs4 as bs
import urllib.request
import re
import PyPDF2
from PyPDF2 import PdfFileReader
import nltk
import fasttext
model = fasttext.load_model('data/fastText/lid.176.ftz')
import string

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 2 Preprocessing</h1>
</div>

In [2]:
#step 2. get the stop words from .txt document
def get_stop_words():
    return  open('data/data/stopwords.txt', encoding='utf-8').read().split()
print(get_stop_words())

['\ufeffወ', 'በጣም', 'ወደ', 'ሁሉ', 'ነው', 'ነዉ', 'ላይ', 'እንደ', 'ግን', 'ዘንድ', 'ነበር', 'ጋር', 'ነገር', 'ስለ', 'እንዲህ', 'እኔ', 'አለ', 'ውስጥ', 'ደግሞ', 'ብሎ', 'አለው', 'ምን', 'ድረስ', 'እርሱም', 'ነበረ', 'ይህ', 'እስከ', 'አንተ', 'ይላል', 'በኋላ', 'እርሱ', 'ሆነ', 'እነሆ', 'ወይም', 'ይህን', 'መካከል', 'ናቸው', 'ይሆናል', 'ስለዚህ', 'ነበሩ', 'እንጂ', 'ብዙ', 'እናንተ', 'እኔም', 'ብቻ', 'አላቸው', 'በዚህ', 'ያለ', 'እና', 'ደስ', 'ማን', 'እንዴት', 'አሉ', 'አሁን', 'እንዲሁ', 'በዚያም', 'እነርሱም', 'በዚያ', 'ከፍ', 'አይደለም', 'ምዕራፍ', 'እጅግ', 'ነውና', 'ብለው', 'ይሁን', 'ሌላ', 'እንግዲህ', 'ሳይሆን', 'በመሆኑም', 'አርስዎ', 'አንደነበር', 'አም', 'ዓም']


In [3]:
def remove_punc_and_special_chars(text): 
    normalized_text = re.sub('[\!\@\#\$\%\^\«\»\&\*\(\)\…\[\]\{\}\;\“\”\›\’\‘\"\'\:\,\.\‹\/\<\>\?\\\\|\`\´\~\-\=\+\፡\፤\;\፦\፥\፧\፨\፠\፣]', '',text) 
    return normalized_text
def sentence_ending(text):
    ending = re.sub('[\።]',',',text)
    return ending
def remove_ascii_and_numbers(text):
    rm_num_and_ascii=re.sub('[A-Za-z0-9]','',text)
    return re.sub('[\'\u1369-\u137C\']+','',rm_num_and_ascii)

In [13]:
def stemming(text):
    word=re.sub('[ጄኔራሎች]','ጀነራል',text)
    return word
print(stemming("ጄኔራሎች"))

ጀነራልጀነራልጀነራልጀነራልጀነራል


In [14]:
def normalize_char_level_missmatch(text):
    rep1=re.sub('[ሃኅኃሐሓኻ]','ሀ',text)
    rep2=re.sub('[ሑኁዅ]','ሁ',rep1)
    rep3=re.sub('[ኂሒኺ]','ሂ',rep2)
    rep4=re.sub('[ኌሔዄ]','ሄ',rep3)
    rep5=re.sub('[ሕኅ]','ህ',rep4)
    rep6=re.sub('[ኆሖኾ]','ሆ',rep5)
    rep7=re.sub('[ሠ]','ሰ',rep6)
    rep8=re.sub('[ሡ]','ሱ',rep7)
    rep9=re.sub('[ሢ]','ሲ',rep8)
    rep10=re.sub('[ሣ]','ሳ',rep9)
    rep11=re.sub('[ሤ]','ሴ',rep10)
    rep12=re.sub('[ሥ]','ስ',rep11)
    rep13=re.sub('[ሦ]','ሶ',rep12)
    rep14=re.sub('[ዓኣዐ]','አ',rep13)
    rep15=re.sub('[ዑ]','ኡ',rep14)
    rep16=re.sub('[ዒ]','ኢ',rep15)
    rep17=re.sub('[ዔ]','ኤ',rep16)
    rep18=re.sub('[ዕ]','እ',rep17)
    rep19=re.sub('[ዖ]','ኦ',rep18)
    rep20=re.sub('[ጸ]','ፀ',rep19)
    rep21=re.sub('[ጹ]','ፁ',rep20)
    rep22=re.sub('[ጺ]','ፂ',rep21)
    rep23=re.sub('[ጻ]','ፃ',rep22)
    rep24=re.sub('[ጼ]','ፄ',rep23)
    rep25=re.sub('[ጽ]','ፅ',rep24)
    rep26=re.sub('[ጾ]','ፆ',rep25)
    return rep26
print(normalize_char_level_missmatch("ሐብታሙ የ ሥራ ሠዉ ነው"))

ሀብታሙ የ ስራ ሰዉ ነው


In [29]:
def clean_text(text):
    text = text.strip()
    text = remove_punc_and_special_chars(text)
    text = remove_ascii_and_numbers(text)
    text = sentence_ending(text)
    text = normalize_char_level_missmatch(text)
    return text
print(clean_text("ዐመታዊ የብሔር ብሔረሠቦች ቀን/Day 20"))

አመታዊ የብሄር ብሄረሰቦች ቀን 


<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 3 Getting File From PDF, Txt, Wikipedia</h1>
</div>

In [30]:
#Step 4. Define functions for Reading Input Text
#Function to Read .txt File and return its Text
def file_text(filepath):
    with open(filepath, encoding="utf8") as f:
        text = f.read().replace("\n", '')
        return text

In [31]:
#Function to Read PDF File and return its Text
def pdfReader(pdf_path):
    
    with open(pdf_path, 'rb') as pdfFileObject:
        pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
        count = pdfReader.numPages
        print("\nTotal Pages in pdf = ", count)
        
        c = 'Y'
        start_page = 0
        end_page = count-1
        c = input("Do you want to read entire pdf ?[Y]/N  :  ")
        if c == 'N' or c == 'n' :
            start_page  = int(input("Enter start page number (Indexing start from 0) :  "))
            end_page = int(input(f"Enter end page number (Less than {count}) : "))
            
            if start_page <0 or start_page >= count:
                print("\nInvalid Start page given")
                sys.exit()
                
            if end_page <0 or end_page >= count:
                print("\nInvalid End page given")
                sys.exit()
                
        for i in range(start_page,end_page+1):
            page = pdfReader.getPage(i)
        return page.extractText()
    page.close()

In [32]:
#Function to Read wikipedia page url and return its Text   
def wiki_text(url):
    scrap_data = urllib.request.urlopen(url)
    article = scrap_data.read()
    parsed_article = bs.BeautifulSoup(article,'lxml')
    
    paragraphs = parsed_article.find_all('p')
    article_text = ""
    
    for p in paragraphs:
        article_text += p.text
    
    #Removing all unwanted characters
    article_text = re.sub(r'\[[0-9]*\]', '', article_text)
    return article_text

In [35]:
#Step 5. Getting Text 
input_text_type = int(input("Select one way of inputting your text :- \n \
                            1. Type your Text(or Copy-Paste)\n \
                            2. Load from .txt file\n \
                            3. Load from .pdf file\n \
                            4. From Wikipedia Page URL\n\n"))
if input_text_type == 1:
    text = clean_text(input(u"Enter your text : \n\n"))
elif input_text_type == 2:
    txt_path = input("Enter file path :  ")
    text = clean_text(file_text(txt_path))  
elif input_text_type == 3:
    file_path = input("Enter file path :  ")
    text = clean_text(pdfReader(file_path))    
elif input_text_type == 4:
    wiki_url = input("Enter Wikipedia URL to load Article : ")
    text = clean_text(wiki_text(wiki_url))    
else:
    print("Sorry! Wrong Input, Try Again.")
#Counting number of words in original article
original_words = text.split()
original_words = [w for w in original_words]
num_words_in_original_text = len(original_words)
sentences = list(text.split(",")) #Extracting all sentences from the text in a list with ::
total_sentences = len(sentences)
print("Original Document: ",sentences)
print("\n")
print("Number of Sentences in Original Document: ",total_sentences)

Select one way of inputting your text :- 
                             1. Type your Text(or Copy-Paste)
                             2. Load from .txt file
                             3. Load from .pdf file
                             4. From Wikipedia Page URL

2
Enter file path :  am.txt
Original Document:  ['የቀድሞ የደርግ መንግስት አገልጋይ የነበርኩ ሻምበል አስተዋይ ለጓድ ሻምበል ተስፋዬ ርስቴ ከ አም ወዲህ በደርግ ስርአት ስለነበረው የአገር መከላከያ ሚኒስቴር መቤት አንዳንድ መኮንኖች በወቅቱ ስለነበረው የመለዮ ለባሹ ሚናና ምን አይነት አመራር ይሰጥ እንደነበር ገመናቸውን ጭምር በመፅሀፍ መልክ ፅፈው ህብረተስቡ እንዲያነበው ማድረጋቸው ይታወሳል', '  የጄኔራል አማን ሚካኤል አንዶም አሟሟትን በሚመለከት ሻምበል ተስፋዬ ርስቴ ጄኔራል ሚካኤል አንዶምን በሚመለከት እንዲህ ብለዋል', '  ይህ በሁለቱ ጄኔራሎች መሀል የተደረገ የስልክ ልውውጥ ጄኔራል ግዛው በላይነህ በደርግ እስከመጨረሻው የተከበሩ እንዲሆኑ የጄኔራል አማን ሚካኤል አንዶም መጨረሻ ቅፅበታዊ እንዲሆን የፖለቲካ ሁኔታው አስገደደ ገፅ ይህን መልእከት የጠለፈው ልዩ ካቢኔ በወታደራዊ አስተዳደር ወቅት የውጭ አገሮች ጥናትና ምርምር ድርጅት የሚባለው የውጭ መረጃ ቢሮ ነው ገፅ ይላል አስተያየት ውድ ሻምበል ተስፋዬ ርስቴ ጄኔራል አማን ሚካኤል አንዶም በ መጀመሪያ ወራት የአገሪቱ ርእሰ ብሄር በሆኑ በጥቂት ወራት ውስጥ የደርግ አባላቱ አገርን ለመምራት ብቃት እንደሌላቸው አገሪቱም በነሱ ተመርታ ወደጥፋት መሄድ የለባትም ማለታ

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 4 Calculate frequency of word in a Sentence</h1>
</div>

In [36]:
#Step 6: Function to calculate frequency of word in each sentence
def frequency_matrix(sentences):
    freq_matrix = {}
    stopWords = get_stop_words()
    for sent in sentences:
        freq_table = {} #dictionary with 'words' as key and their 'frequency' as value
        words = sent.split()
        for word in words:
            if word in stopWords: #Reject stopwords
                continue;
            else:
                if word in freq_table:
                    freq_table[word] += 1
                else:
                    freq_table[word] = 1
                freq_matrix[sent[:15]] = freq_table
    return freq_matrix     
freq_matrix = frequency_matrix(sentences)
print("Word Frequency Matrix",freq_matrix)

Word Frequency Matrix {'የቀድሞ የደርግ መንግስት': {'የቀድሞ': 1, 'የደርግ': 1, 'መንግስት': 1, 'አገልጋይ': 1, 'የነበርኩ': 1, 'ሻምበል': 2, 'አስተዋይ': 1, 'ለጓድ': 1, 'ተስፋዬ': 1, 'ርስቴ': 1, 'ከ': 1, 'ወዲህ': 1, 'በደርግ': 1, 'ስርአት': 1, 'ስለነበረው': 2, 'የአገር': 1, 'መከላከያ': 1, 'ሚኒስቴር': 1, 'መቤት': 1, 'አንዳንድ': 1, 'መኮንኖች': 1, 'በወቅቱ': 1, 'የመለዮ': 1, 'ለባሹ': 1, 'ሚናና': 1, 'አይነት': 1, 'አመራር': 1, 'ይሰጥ': 1, 'እንደነበር': 1, 'ገመናቸውን': 1, 'ጭምር': 1, 'በመፅሀፍ': 1, 'መልክ': 1, 'ፅፈው': 1, 'ህብረተስቡ': 1, 'እንዲያነበው': 1, 'ማድረጋቸው': 1, 'ይታወሳል': 1}, '  የጄኔራል አማን ሚካኤ': {'የጄኔራል': 1, 'አማን': 1, 'ሚካኤል': 2, 'አንዶም': 1, 'አሟሟትን': 1, 'በሚመለከት': 2, 'ሻምበል': 1, 'ተስፋዬ': 1, 'ርስቴ': 1, 'ጄኔራል': 1, 'አንዶምን': 1, 'ብለዋል': 1}, '  ይህ በሁለቱ ጄኔራሎች': {'በሁለቱ': 1, 'ጄኔራሎች': 1, 'መሀል': 1, 'የተደረገ': 1, 'የስልክ': 1, 'ልውውጥ': 1, 'ጄኔራል': 2, 'ግዛው': 1, 'በላይነህ': 1, 'በደርግ': 1, 'እስከመጨረሻው': 1, 'የተከበሩ': 1, 'እንዲሆኑ': 1, 'የጄኔራል': 1, 'አማን': 2, 'ሚካኤል': 2, 'አንዶም': 2, 'መጨረሻ': 1, 'ቅፅበታዊ': 1, 'እንዲሆን': 1, 'የፖለቲካ': 1, 'ሁኔታው': 1, 'አስገደደ': 1, 'ገፅ': 2, 'መልእከት': 1, 'የጠለፈው': 1, 'ልዩ': 1, 'ካቢኔ': 1, 'በወታደራዊ': 1, 'አስተዳደር': 1, 'ወቅት': 1, 

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 5 Calculate Term Frequency of each word</h1>
</div>

In [37]:
#Step 7: Function to calculate Term Frequency(TF) of each word
#TF(t) = (Number of times term t appears in  document) / (Total number of words in the document)
def tf_matrix(freq_matrix):
    tf_matrix = {}
    for sent, freq_table in freq_matrix.items():
        tf_table = {}  #dictionary with 'word' itself as a key and its TF as value
        total_words_in_sentence = len(freq_table)
        for word, count in freq_table.items():
            tf_table[word] = count / total_words_in_sentence
        tf_matrix[sent] = tf_table
    return tf_matrix
tf_matrix = tf_matrix(freq_matrix)
print("Term Frequency in the document: ",tf_matrix)

Term Frequency in the document:  {'የቀድሞ የደርግ መንግስት': {'የቀድሞ': 0.02631578947368421, 'የደርግ': 0.02631578947368421, 'መንግስት': 0.02631578947368421, 'አገልጋይ': 0.02631578947368421, 'የነበርኩ': 0.02631578947368421, 'ሻምበል': 0.05263157894736842, 'አስተዋይ': 0.02631578947368421, 'ለጓድ': 0.02631578947368421, 'ተስፋዬ': 0.02631578947368421, 'ርስቴ': 0.02631578947368421, 'ከ': 0.02631578947368421, 'ወዲህ': 0.02631578947368421, 'በደርግ': 0.02631578947368421, 'ስርአት': 0.02631578947368421, 'ስለነበረው': 0.05263157894736842, 'የአገር': 0.02631578947368421, 'መከላከያ': 0.02631578947368421, 'ሚኒስቴር': 0.02631578947368421, 'መቤት': 0.02631578947368421, 'አንዳንድ': 0.02631578947368421, 'መኮንኖች': 0.02631578947368421, 'በወቅቱ': 0.02631578947368421, 'የመለዮ': 0.02631578947368421, 'ለባሹ': 0.02631578947368421, 'ሚናና': 0.02631578947368421, 'አይነት': 0.02631578947368421, 'አመራር': 0.02631578947368421, 'ይሰጥ': 0.02631578947368421, 'እንደነበር': 0.02631578947368421, 'ገመናቸውን': 0.02631578947368421, 'ጭምር': 0.02631578947368421, 'በመፅሀፍ': 0.02631578947368421, 'መልክ': 0.02631

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 6 Find Sentences per Word or Sentence Containing the word</h1>
</div>

In [22]:
#Step 8. Function to find how many sentences contain a 'word' or sentences containing that word as value)
def sentences_per_words(freq_matrix):
    sent_per_words = {}
    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in sent_per_words:
                sent_per_words[word] += 1
            else:
                sent_per_words[word] = 1
    return sent_per_words
num_sent_per_words = sentences_per_words(freq_matrix)
print("Sentence Containing the Word: ",num_sent_per_words)

Sentence Containing the Word:  {'የቀድሞ': 1, 'የደርግ': 2, 'መንግስት': 1, 'አገልጋይ': 1, 'የነበርኩ': 1, 'ሻምበል': 10, 'አስተዋይ': 1, 'ለጓድ': 1, 'ተስፋዬ': 10, 'ርስቴ': 9, 'ከ': 2, 'ወዲህ': 1, 'በደርግ': 3, 'ስርአት': 3, 'ስለነበረው': 1, 'የአገር': 1, 'መከላከያ': 1, 'ሚኒስቴር': 1, 'መቤት': 1, 'አንዳንድ': 1, 'መኮንኖች': 2, 'በወቅቱ': 3, 'የመለዮ': 1, 'ለባሹ': 1, 'ሚናና': 1, 'አይነት': 1, 'አመራር': 1, 'ይሰጥ': 1, 'እንደነበር': 4, 'ገመናቸውን': 1, 'ጭምር': 1, 'በመፅሀፍ': 1, 'መልክ': 1, 'ፅፈው': 1, 'ህብረተስቡ': 1, 'እንዲያነበው': 1, 'ማድረጋቸው': 1, 'ይታወሳል': 1, 'የጄኔራል': 2, 'አማን': 3, 'ሚካኤል': 3, 'አንዶም': 3, 'አሟሟትን': 1, 'በሚመለከት': 3, 'ጄኔራል': 3, 'አንዶምን': 1, 'ብለዋል': 3, 'በሁለቱ': 1, 'ጄኔራሎች': 1, 'መሀል': 1, 'የተደረገ': 1, 'የስልክ': 1, 'ልውውጥ': 1, 'ግዛው': 1, 'በላይነህ': 1, 'እስከመጨረሻው': 1, 'የተከበሩ': 1, 'እንዲሆኑ': 1, 'መጨረሻ': 1, 'ቅፅበታዊ': 1, 'እንዲሆን': 1, 'የፖለቲካ': 1, 'ሁኔታው': 1, 'አስገደደ': 1, 'ገፅ': 1, 'መልእከት': 1, 'የጠለፈው': 1, 'ልዩ': 2, 'ካቢኔ': 2, 'በወታደራዊ': 1, 'አስተዳደር': 1, 'ወቅት': 1, 'የውጭ': 3, 'አገሮች': 4, 'ጥናትና': 4, 'ምርምር': 4, 'ድርጅት': 4, 'የሚባለው': 1, 'መረጃ': 1, 'ቢሮ': 1, 'አስተያየት': 2, 'ውድ': 1, 'በ': 1, 'መጀመሪያ': 1, 'ወራት': 1, 'የአገሪቱ': 1, 

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 7 Calculate Inverse Document Frequency for a word</h1>
</div>

In [23]:
#Step 9. Function to calculate Inverse Document frequency(IDF) for each word
#IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
def idf_matrix(freq_matrix, sent_per_words, total_sentences):
    idf_matrix = {}
    for sent, f_table in freq_matrix.items():
        idf_table = {}
        for word in f_table.keys():
            idf_table[word] = math.log10(total_sentences / float(sent_per_words[word]))
        idf_matrix[sent] = idf_table
    return idf_matrix
idf_matrix = idf_matrix(freq_matrix, num_sent_per_words, total_sentences)
print("IDF matrix: ",idf_matrix)

IDF matrix:  {'የቀድሞ የደርግ መንግስት': {'የቀድሞ': 1.0413926851582251, 'የደርግ': 0.7403626894942439, 'መንግስት': 1.0413926851582251, 'አገልጋይ': 1.0413926851582251, 'የነበርኩ': 1.0413926851582251, 'ሻምበል': 0.04139268515822508, 'አስተዋይ': 1.0413926851582251, 'ለጓድ': 1.0413926851582251, 'ተስፋዬ': 0.04139268515822508, 'ርስቴ': 0.0871501757189002, 'ከ': 0.7403626894942439, 'ወዲህ': 1.0413926851582251, 'በደርግ': 0.5642714304385625, 'ስርአት': 0.5642714304385625, 'ስለነበረው': 1.0413926851582251, 'የአገር': 1.0413926851582251, 'መከላከያ': 1.0413926851582251, 'ሚኒስቴር': 1.0413926851582251, 'መቤት': 1.0413926851582251, 'አንዳንድ': 1.0413926851582251, 'መኮንኖች': 0.7403626894942439, 'በወቅቱ': 0.5642714304385625, 'የመለዮ': 1.0413926851582251, 'ለባሹ': 1.0413926851582251, 'ሚናና': 1.0413926851582251, 'አይነት': 1.0413926851582251, 'አመራር': 1.0413926851582251, 'ይሰጥ': 1.0413926851582251, 'እንደነበር': 0.43933269383026263, 'ገመናቸውን': 1.0413926851582251, 'ጭምር': 1.0413926851582251, 'በመፅሀፍ': 1.0413926851582251, 'መልክ': 1.0413926851582251, 'ፅፈው': 1.0413926851582251, 'ህብረተስቡ':

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 8 Calculate TF * IDF </h1>
</div>

In [24]:
#Step 10. Function to calculate Tf-Idf score of each word
def tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}
    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):
        tf_idf_table = {}
       #word1 and word2 are same
        for (word1, tf_value), (word2, idf_value) in zip(f_table1.items(), f_table2.items()):  
            tf_idf_table[word1] = float(tf_value * idf_value)
        tf_idf_matrix[sent1] = tf_idf_table
    return tf_idf_matrix
tf_idf_matrix = tf_idf_matrix(tf_matrix, idf_matrix)
print("TF IDF Matrix: ",tf_idf_matrix)

TF IDF Matrix:  {'የቀድሞ የደርግ መንግስት': {'የቀድሞ': 0.027405070662058554, 'የደርግ': 0.019483228670901152, 'መንግስት': 0.027405070662058554, 'አገልጋይ': 0.027405070662058554, 'የነበርኩ': 0.027405070662058554, 'ሻምበል': 0.002178562376748688, 'አስተዋይ': 0.027405070662058554, 'ለጓድ': 0.027405070662058554, 'ተስፋዬ': 0.001089281188374344, 'ርስቴ': 0.0022934256768131632, 'ከ': 0.019483228670901152, 'ወዲህ': 0.027405070662058554, 'በደርግ': 0.014849248169435855, 'ስርአት': 0.014849248169435855, 'ስለነበረው': 0.05481014132411711, 'የአገር': 0.027405070662058554, 'መከላከያ': 0.027405070662058554, 'ሚኒስቴር': 0.027405070662058554, 'መቤት': 0.027405070662058554, 'አንዳንድ': 0.027405070662058554, 'መኮንኖች': 0.019483228670901152, 'በወቅቱ': 0.014849248169435855, 'የመለዮ': 0.027405070662058554, 'ለባሹ': 0.027405070662058554, 'ሚናና': 0.027405070662058554, 'አይነት': 0.027405070662058554, 'አመራር': 0.027405070662058554, 'ይሰጥ': 0.027405070662058554, 'እንደነበር': 0.011561386679743752, 'ገመናቸውን': 0.027405070662058554, 'ጭምር': 0.027405070662058554, 'በመፅሀፍ': 0.027405070662058554,

<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 9 Score each sentence based on tf-idf</h1>
</div>

In [25]:
#Step 11. Function to rate every sentence with some score calculated on basis of Tf-Idf
def score_sentences(tf_idf_matrix):
    sentenceScore = {}
    for sent, f_table in tf_idf_matrix.items():
        total_tfidf_score_per_sentence = 0
        total_words_in_sentence = len(f_table)
        for word, tf_idf_score in f_table.items():
            total_tfidf_score_per_sentence += tf_idf_score
        if total_words_in_sentence != 0:
            sentenceScore[sent] = total_tfidf_score_per_sentence / total_words_in_sentence
    return sentenceScore
sentence_scores = score_sentences(tf_idf_matrix)
print("Sentence Score: ", sentence_scores)

Sentence Score:  {'የቀድሞ የደርግ መንግስት': 0.024075450937957602, '  የጄኔራል አማን ሚካኤ': 0.05213371562051768, '  ይህ በሁለቱ ጄኔራሎች': 0.013360001605868008, '  ጄኔራል አማን ሚካኤል': 0.02363360206788651, '  ቀድሞ ስሙ ልዩ ካቢኔ': 0.04124236439557996, '  ሻምበል ተስፋዬ ርስቴ': 0.034736217886554996, '  ሻምበል አመሀ አበበን': 0.11242950932596786, '  አስተያየት ሻምበል ተ': 0.028762784809481374, '  ከላይ ስማቸው የተገለ': 0.025527077899089486, '  እርስዎ ሻምበል ተስፋ': 0.07259908237315525, '  ሻምበል በኋላም ኮሎኔ': 0.031438606993161214}


<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 10 Calculate Average Score of the Sentence</h1>
</div>

In [26]:
#Step 12. Function Calculating average sentence score 
def average_score(sentence_score):
    total_score = 0
    for sent in sentence_score:
        total_score += sentence_score[sent]
    average_sent_score = (total_score / len(sentence_score))
    return average_sent_score
threshold = average_score(sentence_scores)
print("Average Score of The Sentence : ", threshold)

Average Score of The Sentence :  0.04181258308320181


<div style = "background-color:green">
    <h1 style="color:white;padding:10px;">Step 11 Create Summary of sentence score greater than the average</h1>
</div>

In [38]:
#Step 13. Function to return summary of article
def create_summary(sentences, sentence_score, threshold):
    summary = ''
    for sentence in sentences:
        if sentence[:15] in sentence_score and sentence_score[sentence[:15]] >= (threshold):
            summary += " " + sentence+'።'
    return summary
summary = create_summary(sentences, sentence_scores, 0.9*threshold)
print("*"*55,"Original Text","*"*55)
print("\n",text,"\n\n")
print("*"*54,"Summarized Text","*"*54)
print("\n",summary,"\n\n")
print("Total words in original article = ", num_words_in_original_text)
print("Total words in summarized article = ", len(summary.split()))


******************************************************* Original Text *******************************************************

 የቀድሞ የደርግ መንግስት አገልጋይ የነበርኩ ሻምበል አስተዋይ ለጓድ ሻምበል ተስፋዬ ርስቴ ከ አም ወዲህ በደርግ ስርአት ስለነበረው የአገር መከላከያ ሚኒስቴር መቤት አንዳንድ መኮንኖች በወቅቱ ስለነበረው የመለዮ ለባሹ ሚናና ምን አይነት አመራር ይሰጥ እንደነበር ገመናቸውን ጭምር በመፅሀፍ መልክ ፅፈው ህብረተስቡ እንዲያነበው ማድረጋቸው ይታወሳል,  የጄኔራል አማን ሚካኤል አንዶም አሟሟትን በሚመለከት ሻምበል ተስፋዬ ርስቴ ጄኔራል ሚካኤል አንዶምን በሚመለከት እንዲህ ብለዋል,  ይህ በሁለቱ ጄኔራሎች መሀል የተደረገ የስልክ ልውውጥ ጄኔራል ግዛው በላይነህ በደርግ እስከመጨረሻው የተከበሩ እንዲሆኑ የጄኔራል አማን ሚካኤል አንዶም መጨረሻ ቅፅበታዊ እንዲሆን የፖለቲካ ሁኔታው አስገደደ ገፅ ይህን መልእከት የጠለፈው ልዩ ካቢኔ በወታደራዊ አስተዳደር ወቅት የውጭ አገሮች ጥናትና ምርምር ድርጅት የሚባለው የውጭ መረጃ ቢሮ ነው ገፅ ይላል አስተያየት ውድ ሻምበል ተስፋዬ ርስቴ ጄኔራል አማን ሚካኤል አንዶም በ መጀመሪያ ወራት የአገሪቱ ርእሰ ብሄር በሆኑ በጥቂት ወራት ውስጥ የደርግ አባላቱ አገርን ለመምራት ብቃት እንደሌላቸው አገሪቱም በነሱ ተመርታ ወደጥፋት መሄድ የለባትም ማለታቸው አገራችን አሁን ከደረሰችበት እውነታ ጋር ሲመዛዘን ትክክለኛ ትንበያ የተናገሩ አይመስልምን,  ጄኔራል አማን ሚካኤል አንዶም የደርግን ስርአት ለመቀየር የነበረው እቅዳቸው ሊከሽፍ የቻለው በአገር ውስጥ ጉዳይ ሚኒስትር የውጭ አገሮች ጥናትና ምርምር ተብሎ ይጠራ የነበረው ድርጅት ስልክ በመጥለፍ ባስተላለፈ

<div style = "background-color:rgb(100,100,255)">
    <h1 style="color:white;padding:20px;text-align: center">Thank You !!</h1>
</div>